In [5]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import math

#Loading Mnist dataset

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [7]:
print(x_train.shape,x_test.shape,"\n")
print(y_train.shape,y_test.shape,"\n")

(60000, 28, 28) (10000, 28, 28) 

(60000,) (10000,) 



#Doing dimensionality reduction for Training set
#and get the centroid of each sub-image, after splitting the image into 4X4 sub-images

In [8]:
k = 0
j=0
x_axis = 0
y_axis = 0

array_train = np.zeros([60000,32])

for t in range(x_train.shape[0]):
    x_axis = 0
    y_axis = 0
    j = 0
    k = 7
    h = 7
    i = 0
    l = 0
    img = np.zeros([16,7,7])

    for i in range(16):
        if i%4 == 0 and i != 0:
            k += 7
            h = 7
        img[i] = x_train[t][k-7:k,h-7:h]
        h += 7

    for l in range(16):
        for i in range(7):
            for j in range(7):
                x_axis += img[l][i][j]*i

        if sum(sum(img[l])) == 0:
            array_train[t][l] = 0

        else:
            array_train[t][l] = x_axis/sum(sum(img[l]))
        x_axis = 0

    ################################################

    i = 0
    j = 0
    l = 0
    y_axis = 0
    for l in range(16):
        for i in range(7):
            for j in range(7):
                y_axis = y_axis + img[l][j][i]*i

        if sum(sum(img[l])) == 0:
            array_train[t][l+16] = 0
        else:
            array_train[t][l+16] = y_axis/sum(sum(img[l]))
        y_axis = 0

#Doing dimensionality reduction for Testing set
#and get the centroid of each sub-image, after splitting the image into 4X4 sub-images

In [9]:
k = 0
j=0
x_axis = 0
y_axis = 0
array_test = np.zeros([10000,32])
for t in range(x_test.shape[0]):
    x_axis = 0
    y_axis = 0
    j = 0
    i = 0
    l = 0
    img = np.zeros([16,7,7])
    k = 7
    h = 7

    for i in range(16):
        if i%4 == 0 and i != 0:
            k += 7
            h = 7
        img[i] = x_test[t][k-7:k,h-7:h]
        h += 7

    for l in range(16):
        for i in range(7):
            for j in range(7):
                x_axis += img[l][i][j]*i

        if sum(sum(img[l])) == 0:
            array_test[t][l] = 0

        else:
            array_test[t][l] = x_axis/sum(sum(img[l]))
        x_axis = 0

    ################################################

    i = 0
    j = 0
    l = 0
    y_axis = 0

    for l in range(16):
        for i in range(7):
            for j in range(7):
                y_axis = y_axis + img[l][j][i]*i

        if sum(sum(img[l])) == 0:
            array_test[t][l+16] = 0
        else:
            array_test[t][l+16] = y_axis/sum(sum(img[l]))
        y_axis = 0

#Normalize the features

In [10]:
for i in range (array_train.shape[1]):
    array_train[:,i] = (array_train[:,i] - np.mean(array_train[:,i])) / np.std(array_train[:,i])
    array_test[:,i] = (array_test[:,i] - np.mean(array_test[:,i])) / np.std(array_test[:,i])

#Creating validation set

In [11]:
array_val = array_train[:6000,:]
array_train = array_train[6000:,:]
y_val = y_train[:6000]
y_train = y_train[6000:]

In [12]:
array_val.shape, y_val.shape, array_train.shape, y_train.shape

((6000, 32), (6000,), (54000, 32), (54000,))

#implementing sigmoid activation function

In [13]:
def sigmoid(array):
    return 1/(1+np.exp(-array.copy(),dtype = np.float128))

#Defining Softmax activation function

In [14]:
def softmax(array):
    post_values = array.copy()
    post_values = np.exp(post_values, dtype = np.float128)
    sum_of_output_values = np.sum(post_values.copy(),axis=1)
    for i in range(post_values.shape[1]):
        post_values[:,i] /= sum_of_output_values
    return post_values

#Defining function convert the integer number to a one-hot dimension with respect to the number of classes in the set

In [15]:
def dimensions(y_train):
    real_y = np.zeros(shape=(1,10))
    real_y[:,y_train] = 1
    return real_y

#Defining function computes the gradient **rate of change** of the loss values with respect to the weights

In [16]:
def sigmoid_gradient(array):
    return np.multiply(array,1-array)

#Defining the loss function **Cross entropy** for classification

In [17]:
def loss_function(x,y,parameters, reg_lambda):
  h_theta = x
  real_y = dimensions_10(y)
  m = h_theta.shape[0]

  for i in range(parameters.shape[0]-1):
    h_theta = sigmoid(np.dot(h_theta,parameters[i].parameter_matrix))

  h_theta = softmax(np.dot(h_theta,parameters[parameters.shape[0]-1].parameter_matrix))
  cost = np.multiply(real_y.copy(),np.log(h_theta)) + np.multiply(1-real_y.copy(),np.log(1-h_theta))
  regularized_cost = 0
  for i in range(parameters.shape[0]):
    regularized_cost += np.sum((np.square(np.array(parameters[i].parameter_matrix))))
  regularized_cost *= (reg_lambda/(2*m))
  cost = np.sum(cost)
  cost /= m
  cost = -cost
  cost += regularized_cost
  print("cost is ", end="")
  return cost

#Defining the matrix class which is desired for initalize the weight matrix

In [18]:
class matrix:
    def __init__(self, current_layer, next_layer):
        self.current_layer = current_layer
        self.next_layer = next_layer
        self.parameter_matrix = np.random.normal(size=(self.current_layer,self.next_layer))

    def get_parameters(self):
        return self.parameter_matrix

#Defining the Neural Network class to build the desired architecture

In [19]:
class Neural_Network:
    def __init__(self, layers, neurons):
        self.layers = layers
        self.neurons = neurons
        self.parameter_matrices = np.empty(shape=(self.layers - 1,),dtype=matrix)
        self.outputs = np.zeros(shape=(self.layers,),dtype=matrix)

    def preparing(self, train):
        data = train.copy()
        self.parameter_matrices[0] = matrix(data.shape[1], self.neurons)
        self.parameter_matrices[self.layers - 2] = matrix(self.neurons, 10)

        for i in range(1,self.layers - 2):
            self.parameter_matrices[i] = matrix(self.neurons, self.neurons)

        return self.parameter_matrices

    def feed_forward(self,train, parameters):
        data = train.copy()
        self.outputs[0] = matrix(data.shape[0], data.shape[1])
        self.outputs[0].parameter_matrix = data.copy()

        for i in range(1,self.layers-1):
            data = sigmoid(np.dot(data,parameters[i-1].parameter_matrix))
            self.outputs[i] = matrix(data.shape[0], data.shape[1])
            self.outputs[i].parameter_matrix = data.copy()

        data = softmax(np.dot(data,parameters[self.layers - 2].parameter_matrix))
        self.outputs[self.layers - 1] = matrix(data.shape[0], data.shape[1])
        self.outputs[self.layers - 1].parameter_matrix = data.copy()
        return parameters.copy(), self.outputs

    def back_propagation(self, outputs, parameters, target, learning_rate, reg_lambda):
        actual_y = dimensions(target)
        thetas = parameters.copy()
        parameter = parameters.copy()
        data = outputs.copy()

        ########################################################
        # for output layer only
        error = np.multiply((actual_y - data[-1].parameter_matrix), sigmoid_gradient(data[-1].parameter_matrix))

        thetas[-1].parameter_matrix += np.dot(np.transpose(data[-2].parameter_matrix), error) * learning_rate
        # for output layer only
        #########################################################

        #########################################################
        # for hidden layers...

        for i in range(data.shape[0] - 2, 0 , -1):
              error = np.dot(error,np.transpose(parameter[i].parameter_matrix))
              error = np.multiply(error, sigmoid_gradient(data[i].parameter_matrix))
              thetas[i-1].parameter_matrix += (np.dot(np.transpose(data[i-1].parameter_matrix), error) + (reg_lambda/data.shape[0])*data[i-1].parameter_matrix) * learning_rate

        return thetas.copy()

#Defining a function convert the target to on-hot dimension

In [20]:
def dimensions_10(y_train):
    real_y = np.zeros(shape=(y_train.shape[0],np.unique(y_train).shape[0]))
    for i in range(0,y_train.shape[0]):
        real_y[i,y_train[i]] = 1
    return real_y

#Defining the accuracy test function

In [21]:
def accuracy(data,target,parameters):
    real_y = dimensions_10(target.copy())
    results = data.copy()
    thetas = parameters.copy()

    for i in range(thetas.shape[0]-1):
        results = sigmoid(np.dot(results, thetas[i].parameter_matrix))

    results = softmax(np.dot(results, thetas[thetas.shape[0]-1].parameter_matrix))
    hypothesis = np.zeros([results.shape[0],results.shape[1]])
    count = 0

    for m in range(0,results.shape[0]):
        index_ = 0
        biggest = results[m,index_]

        for n in range(1,results.shape[1]):
            if biggest < results[m,n]:
                biggest = results[m,n]
                index_ = n

        hypothesis[m,index_] = 1

    for l in range(0,real_y.shape[0]):
        for v in range(0,real_y.shape[1]):
            if real_y[l,v] == 1 and hypothesis[l,v] == 1:
                count = count + 1


    return(count/results.shape[0])*100

#initalize the Neural Network architecture

In [22]:
N = Neural_Network(7,32)

#Defining fit Function for training

In [23]:
def fit(array_train, iterations, learning_rate, reg_lambda):
    thetas = N.preparing(array_train.copy())
    for iterations in range(0,iterations):
        for i in range(0,array_train.shape[0]):
            thetas, outputs = N.feed_forward(array_train[i:i+1,:],thetas)
            thetas = N.back_propagation(outputs,thetas,y_train[i],learning_rate,reg_lambda)

        print("epoch ",iterations+1,": \nLoss function on Training set",loss_function(array_train,y_train,thetas, reg_lambda))
        print("epoch ",iterations+1,": \nLoss function on Validation set",loss_function(array_val,y_val,thetas, reg_lambda))
        print("=========================================================")
    return thetas

In [24]:
thetas = fit(array_train, 20, 0.01, 1e-5)

cost is epoch  1 : 
Loss function on Training set 2.1460886013008783732
cost is epoch  1 : 
Loss function on Validation set 2.1096900097520314065
cost is epoch  2 : 
Loss function on Training set 1.5354730847034725134
cost is epoch  2 : 
Loss function on Validation set 1.4902997556367340589
cost is epoch  3 : 
Loss function on Training set 1.2482825807849426428
cost is epoch  3 : 
Loss function on Validation set 1.2132246403104544073
cost is epoch  4 : 
Loss function on Training set 1.1056036359740782928
cost is epoch  4 : 
Loss function on Validation set 1.0763483650512458605
cost is epoch  5 : 
Loss function on Training set 1.016036709321464725
cost is epoch  5 : 
Loss function on Validation set 0.98880387248501091553
cost is epoch  6 : 
Loss function on Training set 0.94877995057245291554
cost is epoch  6 : 
Loss function on Validation set 0.92386918305088887424
cost is epoch  7 : 
Loss function on Training set 0.89404940393575072855
cost is epoch  7 : 
Loss function on Validation s

#printing the accuracy after training on the training set on **Training set**

In [25]:
print("accuracy of training set is: ", accuracy(array_train,y_train,thetas))

accuracy of training set is:  89.63148148148147


#printing the accuracy after training on the training set on **Test set**

In [26]:
print("accuracy on test set: ", accuracy(array_test,y_test,thetas))

accuracy on test set:  89.32


#defining a function preparing for prediction

In [27]:
def preparing(result,parameters):
    results = result.copy()
    thetas = parameters.copy()
    for i in range(thetas.shape[0]-1):
        results = sigmoid(np.dot(results, thetas[i].parameter_matrix))

    results = softmax(np.dot(results, thetas[thetas.shape[0]-1].parameter_matrix))

    post_result = np.zeros(shape=(results.shape[0],))
    for i in range(results.shape[0]):
        index = np.argmax(results[i,:])
        post_result[i]=index
    return post_result

#See the Score

In [28]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

In [29]:
results = preparing(array_test,thetas)

#Confusion matrix

In [30]:
confusion_matrix(y_test,results)

array([[ 957,    0,    4,    5,    1,    6,    4,    3,    0,    0],
       [   0, 1099,    5,    6,    5,    4,    3,    3,    7,    3],
       [  11,    3,  942,   21,    6,    2,   10,   11,   24,    2],
       [  12,    8,   31,  871,    2,   35,    5,   13,   26,    7],
       [   0,    3,    8,    4,  893,    5,   11,    5,   12,   41],
       [  13,    5,    1,   80,    8,  722,   17,    2,   36,    8],
       [  12,    1,    9,    1,   10,   18,  898,    5,    4,    0],
       [   2,    5,   18,    7,   21,    2,    4,  888,   10,   71],
       [  14,    6,   14,   50,   23,   42,   14,    5,  779,   27],
       [   7,    6,    2,    7,   65,    9,    0,   17,   13,  883]])

#The accuracy score

In [31]:
accuracy_score(y_test,results)

0.8932

#F1 score

In [32]:
f1_score(y_test,results,average='micro')

0.8932

#Precision score

In [33]:
precision_score(y_test,results,average='micro')

0.8932

#Recall score

In [34]:
recall_score(y_test,results,average='micro')

0.8932